In [105]:
import numpy as np
import random
from random import *
import csv
import pandas as pd
from pprint import pprint

In [106]:
# parameters
N = 3 # of experts
eps = 0.13 #epsilon
T = 61 #total number of time peridos



# notations
# w: np array of expert weights
# f: np array of predictions made by experts
# t: time period from 1...T
# x: decision
# y: outcome of decision x
# M: np array of # mistakes made by experts
# M_wma: total mistakes made by WMA

In [137]:
# initialize weight
w = np.ones(N)
M = np.zeros(N)
rM = np.zeros(N)
M_wma = 0
M_rwma = 0
q = 0
totM_rwma = []
totM_wma = []
totL10 = []
totL01 = []
totrL10 = []
totrL01 = []
# iterations
for x in range(1000):
    M_rwma1 = 0
    rL10 = 0
    rL01 = 0
    w1 = np.ones(N)
    M1 = np.zeros(N)
    rM1 = np.zeros(N)
    losses10 = 0
    losses01 = 0
    with open('C:/Users/james/Desktop/isye4803GUP/Hw1_Hauth/temperature_data.csv', newline='') as csvfile:
        experts = csv.reader(csvfile, delimiter=',')
        next(experts)
        
        for row in experts:
            t = row[0]
    
            # experts make predictions
            f = row[1:1+N]
            for i in range(0, len(f)):
                f[i] = float(f[i]) 
                if f[i] >70:
                    f[i] = 1
                else:
                    f[i] = 0
             
            
            # tally the weighted probabilities of {0,1} predictions
            rprobs = [w1[0]/w1.sum(), w1[1]/w1.sum(), w1[2]/w1.sum()]
            rone = np.dot(rprobs,f)
            rzero = 1 - rone
            rx = choices([0,1], weights = [rzero, rone], k = 1) #make randomly based decision
    
            # reveal outcome
            y = float(row[4])
            if y > 70:
                y=1
            else:
                y = 0
            M_rwma1 += (rx[0] != y)
            if (rx[0] != y) and (rx[0] == 0):
                rL01+=1
            elif (rx[0] != y) and (rx[0] == 1):
                rL10 += 1
            # weight update
            for i in range(N):
                w1[i] = w1[i]*(1 - eps)**(f[i]!=y) # penalize weight if made wrong prediction
                M1[i] = M1[i] + (f[i] != y)
                rM1[i] = rM1[i] + (f[i] != y)
            df = pd.DataFrame({"A_round": t,
                              "weather.com weight": w[0],
                              "accuweather.com weight": w[1],
                              "wunderground.com weight": w[2],
                              "weather.com mistakes": M[0],
                              "accuweather.com mistakes": M[1],
                              "wunderground.com mistakes": M[2]}, index = range(1,2))
        totM_rwma.append(M_rwma1)
        totrL10.append(rL10)
        totrL01.append(rL01)
with open('C:/Users/james/Desktop/isye4803GUP/Hw1_Hauth/temperature_data.csv', newline='') as csvfile:
        experts = csv.reader(csvfile, delimiter=',')
        next(experts)
        L10= 0
        L01 = 0
        for row in experts:
            t = row[0]
            print('round: ', t)
            print('round start expert weights: ', w)
            print('round start mistakes: ', M)
    
            # experts make predictions
            f = row[1:1+N]
            for i in range(0, len(f)):
                f[i] = float(f[i]) 
                if f[i] >70:
                    f[i] = 1
                else:
                    f[i] = 0
            print(f)
            # tally weighted sum of {0,1} predictions  
            one = np.dot(w,f)
            zero = w.sum() - one
            x = 1 if one >= zero else 0 # make majority based decision 
            
            # tally the weighted probabilities of {0,1} predictions
            rprobs = [w[0]/w.sum(), w[1]/w.sum(), w[2]/w.sum()]
            rone = np.dot(rprobs,f)
            rzero = 1 - rone
            rx = choices([0,1], weights = [rzero, rone], k = 1) #make randomly based decision
    
            # reveal outcome
            y = float(row[4])
            if y > 70:
                y=1
            else:
                y = 0
            M_wma += (x != y) #count number of mistakes
            M_rwma += (rx[0] != y)
            if (x != y) and (x == 0):
                L01+=1
            elif (x != y) and (x == 1):
                L10 += 1
            # weight update
            for i in range(N):
                w[i] = w[i]*(1 - eps)**(f[i]!=y) # penalize weight if made wrong prediction
                M[i] = M[i] + (f[i] != y)
                rM[i] = rM[i] + (f[i] != y)
            print('round end random mistakes: ', rM)
            print('round end expert weights: ', w)
            print('round end mistakes: ', M)
            df = pd.DataFrame({"A_round": t,
                              "weather.com weight": w[0],
                              "accuweather.com weight": w[1],
                              "wunderground.com weight": w[2],
                              "weather.com mistakes": M[0],
                              "accuweather.com mistakes": M[1],
                              "wunderground.com mistakes": M[2]}, index = range(1,2))
            totM_wma.append(M_wma)
            totL10.append(L10)
            totL01.append(L01)
            #code for writing data to the csv file. Made as comments after it was intially uploaded so as not to rewrite it
            #every time code is run
'''            if q == 0:
                q = 1
                with open('C:/Users/james/Desktop/isye4803GUP/temperature_data_weights.csv', 'w') as f:
                    df.to_csv(f, header=True)
            else:
                with open('C:/Users/james/Desktop/isye4803GUP/temperature_data_weights.csv', 'a') as f:
                    df.to_csv(f, header=False)
''' 
avgr = sum(totM_rwma)/len(totM_rwma)
avgrL10 = sum(totrL10)/len(totrL10)
avgrL01 = sum(totrL01)/len(totrL01)
print("Average RWMA mistakes made:", avgr)
print("Average mistakes where 1 was chosen: ", avgrL10)
print("Average mistakes where 0 was chosen: ", avgrL01)
print("RWMA regret is: ", avgr-min(avgrL10, avgrL01))
print('WMA mistakes where 1 was chosen: ', L10)
print("WMA mistakes where 0 was chosen: ", L01)
print('WMA regret is: ', M_wma - min(L01,L10))
print('best expert made ', M.min(), ' mistakes')
print('WMA made ', M_wma, ' mistakes')

round:  1
round start expert weights:  [1. 1. 1.]
round start mistakes:  [0. 0. 0.]
[1, 1, 1]
round end random mistakes:  [0. 0. 0.]
round end expert weights:  [1. 1. 1.]
round end mistakes:  [0. 0. 0.]
round:  2
round start expert weights:  [1. 1. 1.]
round start mistakes:  [0. 0. 0.]
[1, 1, 1]
round end random mistakes:  [0. 0. 0.]
round end expert weights:  [1. 1. 1.]
round end mistakes:  [0. 0. 0.]
round:  3
round start expert weights:  [1. 1. 1.]
round start mistakes:  [0. 0. 0.]
[1, 1, 1]
round end random mistakes:  [0. 0. 0.]
round end expert weights:  [1. 1. 1.]
round end mistakes:  [0. 0. 0.]
round:  4
round start expert weights:  [1. 1. 1.]
round start mistakes:  [0. 0. 0.]
[1, 1, 1]
round end random mistakes:  [0. 0. 0.]
round end expert weights:  [1. 1. 1.]
round end mistakes:  [0. 0. 0.]
round:  5
round start expert weights:  [1. 1. 1.]
round start mistakes:  [0. 0. 0.]
[1, 1, 0]
round end random mistakes:  [0. 0. 1.]
round end expert weights:  [1.   1.   0.87]
round end m